In [132]:
import pandas as pd 
import re
import json
import requests as rq
import spacy
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from cleantext import clean
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# Loading and Preparing Data 

In [100]:

df=pd.read_csv(r"C:\Users\abhishek\mypython\ML\NLP\twitter\training_twitter_x_y_train.csv")

In [101]:
traindf=pd.DataFrame(columns=["tweet","sentiment"])
traindf["tweet"]=df["text"]
traindf["sentiment"]=df["airline_sentiment"]

In [134]:
traindf.head(10)

,tweet,sentiment
0,southwestair scheduled morning 2 days fact yes...,negative
1,southwestair seeing workers time time going lo...,positive
2,united flew ord miami great crew service legs ...,positive
3,southwestair dultch97 thats horse radish,negative
4,united flight ord delayed air force flight sbn...,negative
5,united load flying sardine knew pilots 2 hours...,negative
6,jetblue stock response delays frustrating poor...,negative
7,jetblue thatd nice hoping rack miles trip seat...,positive
8,united frankly worse customer service problems...,negative
9,southwestair yeah haha expensive fun destinati...,positive


# Text Pre-Processing

In [104]:
CLEANR = re.compile('<.*?>') 
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words
punc=string.punctuation
lemmatizer = WordNetLemmatizer()

def punc_remover(text):
    for i in text:
        if i in punc:
            text=text.replace(i,'')
    return str(text)

def stopwords_remover(text):
    text=str(text)
    split_text=text.split()
    new_text=[]
    for i in split_text:
        if i not in stopwords:
            new_text.append(i)
    return " ".join(new_text)        

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)            

# def spell_correct(text):
#     text=str(text)
#     textblb=TextBlob(text)
#     return textblb.correct().string 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatizerrr(text):
    text=str(text)
    split_text=text.split()
    new_text=[]
    for i in split_text:
        tag=pos_tag([i])
        lemmatizer.lemmatize(i,pos=get_simple_pos(tag[0][1]))
    return " ".join(split_text) 

def othercleaning(tweet):
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet) #remove links
    tweet = re.sub('@[^\s]+','',tweet) #remove usernames
    tweet = re.sub('[\s]+', ' ', tweet)
    return tweet       


In [105]:
traindf["tweet"]=traindf["tweet"].str.lower()
traindf["tweet"]=traindf["tweet"].apply(punc_remover)
traindf["tweet"]=traindf["tweet"].apply(stopwords_remover)
traindf["tweet"]=traindf["tweet"].apply(remove_emojis)
traindf["tweet"]=traindf["tweet"].apply(cleanhtml)
traindf["tweet"]=traindf["tweet"].apply(lemmatizerrr)
traindf["tweet"]=traindf["tweet"].apply(othercleaning)


In [107]:
X_train, X_test, y_train, y_test = train_test_split(list(traindf["tweet"]),(traindf["sentiment"]) , test_size=0.33, random_state=42)

# Making vocablary

In [129]:
vectorizer= TfidfVectorizer(analyzer='word', max_features=3150, max_df = 0.8, ngram_range=(1,3))
X = vectorizer.fit_transform(X_train)
modelx_train=X.toarray()

In [130]:
xt=vectorizer.fit_transform(X_test)
modelx_test=xt.toarray()

# Applying Model

In [131]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C = 2.1, solver='liblinear', multi_class='auto')
clf.fit(modelx_train,y_train)
clf.score(modelx_test,y_test)

0.5990618101545254